In [1]:
##Libraries
library(tidyverse)
library(tidymodels)
library(embed)
library(themis)
library(vroom)


train <- vroom('/kaggle/input/allstate-claims-severity/train.csv')
test <- vroom('/kaggle/input/allstate-claims-severity/test.csv')
train$loss <- log(train$loss) 

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
rf_recipe <- recipe(loss ~ ., data=train) %>%
  step_lencode_mixed(all_nominal_predictors(), outcome = vars(loss))


#Set up the model
my_mod <- rand_forest(mtry = 5,
                      min_n=2,
                      trees=1000) %>%
  set_engine("ranger") %>%
  set_mode("regression")

In [3]:
## Create a workflow with model & recipe
rf_wf <- workflow() %>%
  add_recipe(rf_recipe) %>%
  add_model(my_mod) %>%
  fit(data=train)

boundary (singular) fit: see help('isSingular')



In [4]:
rf_predictions <- rf_wf %>%
  predict(new_data = test)

Sub1 <- rf_predictions %>%
  bind_cols(test) %>%
  select(id,.pred) %>%
  rename(loss = .pred) %>%
  mutate(loss = exp(loss))

In [5]:
vroom_write(Sub1, file= './submission.csv', delim=',')